# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

In [1]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [2]:
env = gym.make('Pendulum-v0')

random_seed = 10
env.seed(random_seed)

# from agent import Agent
# agent = Agent(state_size=env.observation_space.shape[0], action_size=env.action_space.shape[0], 
#               random_seed=random_seed)

/home/arasdar/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


[10]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [3]:
env.action_space, env.observation_space

(Box(1,), Box(3,))

In [4]:
action_size = env.action_space.shape[0]
state_size = env.observation_space.shape[0]

In [5]:
action_size, state_size

(1, 3)

In [6]:
env.observation_space.low, env.observation_space.high

(array([-1., -1., -8.], dtype=float32), array([1., 1., 8.], dtype=float32))

In [7]:
env.action_space.low, env.action_space.high

(array([-2.], dtype=float32), array([2.], dtype=float32))

### 3. Instantiate DDPG Agent

In [8]:
from agent2 import Agent

agent = Agent(s_size=state_size, a_size=action_size, random_seed=random_seed)

### 4. Train the Agent with DDPG

In [9]:
from collections import deque
import numpy as np
import time
import torch

n_episodes=1000
# n_steps=1000
n_learn=1
# learn_every=20
R_goal=-100 # TOCHECK
Rs_deque = deque(maxlen=100)
meanRs = [] # plot
Rs = [] # plot
dlosses = []
glosses = []

In [ ]:
for i_episode in range(n_episodes):
    
    # # Reset Env
    # env_info = env.reset(train_mode=train_mode)[brain_name]       # reset the environment    
    # S = env_info.vector_observations                   # get the current state (for each agent)
    s = env.reset() # s: state [-1:1, -1:1, -8:8]
    #s_ = np.zeros_like(s) # s_: predicted state [-1:1, -1:1, -8:8]
    #print(s.shape)

    #Rs = np.zeros(num_agents)                            # initialize the score (for each agent)
    R = 0 # R: total reward
    
    glosses_batch = []
    dlosses_batch = []

    #for i_step in range(n_steps):
    while True:
        s = np.reshape(s, [1, -1])
        
        a = agent.act(s) # a=[-2, 2]
        a *= 2
        #print(a.shape)
        
        # Exploration noise
        # noise = np.random.normal(loc=0, scale=0.1, size=[num_agents, action_size])
        # A += noise
        # A = np.clip(a=A, a_min=-1, a_max=1)

        # env_info = env.step(A)[brain_name]            # send all actions to the environment
        # S2 = env_info.vector_observations          # get next state (for each agent)
        # rewards = env_info.rewards                          # get reward (for each agent)
        # dones = env_info.local_done                         # see if episode finished
        #s2: next_state, r: reward, done/terminal, env: environment
        a = np.reshape(a, [-1])
        s2, r, done, _ = env.step(a)
        #print(s2.shape, r, done)

        #s2_, q_ = agent.env(s, a)
        a = np.reshape(a, [1, -1])
        #s2_, _ = agent.env(s, a)
        # print(s2_)
        # s2_ = np.tanh(s2_) # predicted next state [-1:1, -1:1, -8:8]
        # print(s2_)
        # s2_[2] = s2_[2] * 8
        # print(s2_)

        # for s, a, reward, s2, done in zip(S, A, rewards, S2, dones):
        # agent.step(state, action, reward, next_state, done) # send actions to the agent
        agent.step(s, a, r, s2, done)

        R += r                           # update the score (for each agent)
        s = s2                                # roll over the states to the next state

        # if i_step%learn_every == 0:
        #dlosses_Nlearn = []
        #for _ in range(n_learn):
        dloss, _, _ = agent.start_dlearn()
        #print(dloss)
        #dlosses_Nlearn.append(dloss)
        #dlosses_batch.append(np.mean(dlosses_Nlearn))
        dlosses_batch.append(dloss)
            
        #print(len(agent.memory))
        gloss = agent.start_glearn()
        glosses_batch.append(gloss)

        # if np.any(dones):                                   # exit loop if episode finished
        #     break
        if done:
            break

    #meanR = np.mean(Rs)
    #Rs_deque.append(meanR)
    Rs_deque.append(R)
    meanRs.append(np.mean(Rs_deque)) # plot
    Rs.append(R) # plot
    dlosses.append(np.mean(dlosses_batch))
    glosses.append(np.mean(glosses_batch))

    print('\rEpisode {}\tTotal Average Score: {:.2f}\tGloss: {:.2f}, \tDloss: {:.2f}'.format(
           i_episode+1, np.mean(Rs_deque), np.mean(glosses_batch), np.mean(dlosses_batch)))  
    
    if np.mean(Rs_deque) >= R_goal:
        torch.save(agent.g.state_dict(), 'g-pendulum.pth') # pendulum
        torch.save(agent.d.state_dict(), 'd-pendulum.pth') # pendulum
        break

Episode 1	Total Average Score: -883.57	Gloss: 0.00, 	Dloss: 0.00
Episode 2	Total Average Score: -1367.03	Gloss: 0.00, 	Dloss: 0.00
Episode 3	Total Average Score: -1157.02	Gloss: 0.00, 	Dloss: 0.00
Episode 4	Total Average Score: -1088.51	Gloss: 0.00, 	Dloss: 0.00
Episode 5	Total Average Score: -1185.62	Gloss: 0.00, 	Dloss: 0.00
Episode 6	Total Average Score: -1261.48	Gloss: 0.06, 	Dloss: 5.38
Episode 7	Total Average Score: -1298.27	Gloss: 0.15, 	Dloss: 6.38
Episode 8	Total Average Score: -1279.59	Gloss: 0.22, 	Dloss: 6.42
Episode 9	Total Average Score: -1309.05	Gloss: 0.29, 	Dloss: 6.42
Episode 10	Total Average Score: -1313.34	Gloss: 0.36, 	Dloss: 6.49
Episode 11	Total Average Score: -1322.46	Gloss: 0.44, 	Dloss: 6.50
Episode 12	Total Average Score: -1333.62	Gloss: 0.51, 	Dloss: 6.54
Episode 13	Total Average Score: -1351.46	Gloss: 0.59, 	Dloss: 6.62
Episode 14	Total Average Score: -1382.09	Gloss: 0.66, 	Dloss: 6.74
Episode 15	Total Average Score: -1388.02	Gloss: 0.73, 	Dloss: 6.82
Episo

Episode 124	Total Average Score: -1560.32	Gloss: 8.49, 	Dloss: 7.04
Episode 125	Total Average Score: -1558.18	Gloss: 8.56, 	Dloss: 7.04
Episode 126	Total Average Score: -1557.40	Gloss: 8.63, 	Dloss: 7.04
Episode 127	Total Average Score: -1558.07	Gloss: 8.69, 	Dloss: 7.03
Episode 128	Total Average Score: -1562.54	Gloss: 8.75, 	Dloss: 7.02
Episode 129	Total Average Score: -1559.85	Gloss: 8.83, 	Dloss: 7.00
Episode 130	Total Average Score: -1559.42	Gloss: 8.88, 	Dloss: 7.00
Episode 131	Total Average Score: -1565.89	Gloss: 8.96, 	Dloss: 7.00
Episode 132	Total Average Score: -1570.07	Gloss: 9.03, 	Dloss: 7.02
Episode 133	Total Average Score: -1575.25	Gloss: 9.08, 	Dloss: 7.00
Episode 134	Total Average Score: -1573.88	Gloss: 9.18, 	Dloss: 7.00
Episode 135	Total Average Score: -1575.53	Gloss: 9.24, 	Dloss: 7.02
Episode 136	Total Average Score: -1573.49	Gloss: 9.31, 	Dloss: 7.01
Episode 137	Total Average Score: -1576.86	Gloss: 9.38, 	Dloss: 7.00
Episode 138	Total Average Score: -1582.31	Gloss:

Episode 244	Total Average Score: -1435.26	Gloss: 17.96, 	Dloss: 7.30
Episode 245	Total Average Score: -1434.07	Gloss: 18.03, 	Dloss: 7.30
Episode 246	Total Average Score: -1430.18	Gloss: 18.11, 	Dloss: 7.30
Episode 247	Total Average Score: -1432.69	Gloss: 18.19, 	Dloss: 7.29
Episode 248	Total Average Score: -1430.85	Gloss: 18.26, 	Dloss: 7.29
Episode 249	Total Average Score: -1436.97	Gloss: 18.34, 	Dloss: 7.29
Episode 250	Total Average Score: -1437.51	Gloss: 18.41, 	Dloss: 7.29
Episode 251	Total Average Score: -1434.37	Gloss: 18.49, 	Dloss: 7.29
Episode 252	Total Average Score: -1438.95	Gloss: 18.57, 	Dloss: 7.29
Episode 253	Total Average Score: -1436.12	Gloss: 18.64, 	Dloss: 7.28
Episode 254	Total Average Score: -1431.03	Gloss: 18.72, 	Dloss: 7.28
Episode 255	Total Average Score: -1430.32	Gloss: 18.80, 	Dloss: 7.28
Episode 256	Total Average Score: -1423.89	Gloss: 18.87, 	Dloss: 7.28
Episode 257	Total Average Score: -1419.88	Gloss: 18.95, 	Dloss: 7.27
Episode 258	Total Average Score: -

Episode 363	Total Average Score: -1438.63	Gloss: 26.95, 	Dloss: 7.16
Episode 364	Total Average Score: -1438.43	Gloss: 27.03, 	Dloss: 7.17
Episode 365	Total Average Score: -1438.91	Gloss: 27.10, 	Dloss: 7.16
Episode 366	Total Average Score: -1439.56	Gloss: 27.18, 	Dloss: 7.17
Episode 367	Total Average Score: -1437.21	Gloss: 27.25, 	Dloss: 7.16
Episode 368	Total Average Score: -1441.29	Gloss: 27.33, 	Dloss: 7.17
Episode 369	Total Average Score: -1441.56	Gloss: 27.41, 	Dloss: 7.17
Episode 370	Total Average Score: -1441.82	Gloss: 27.48, 	Dloss: 7.17
Episode 371	Total Average Score: -1441.83	Gloss: 27.55, 	Dloss: 7.15
Episode 372	Total Average Score: -1441.59	Gloss: 27.63, 	Dloss: 7.16
Episode 373	Total Average Score: -1441.54	Gloss: 27.70, 	Dloss: 7.17
Episode 374	Total Average Score: -1441.72	Gloss: 27.78, 	Dloss: 7.17
Episode 375	Total Average Score: -1441.94	Gloss: 27.85, 	Dloss: 7.15
Episode 376	Total Average Score: -1440.64	Gloss: 27.93, 	Dloss: 7.17
Episode 377	Total Average Score: -

Episode 482	Total Average Score: -1209.13	Gloss: 35.84, 	Dloss: 6.85
Episode 483	Total Average Score: -1207.49	Gloss: 35.91, 	Dloss: 6.83
Episode 484	Total Average Score: -1204.00	Gloss: 35.98, 	Dloss: 6.85
Episode 485	Total Average Score: -1205.92	Gloss: 36.06, 	Dloss: 6.84
Episode 486	Total Average Score: -1208.09	Gloss: 36.14, 	Dloss: 6.85
Episode 487	Total Average Score: -1205.73	Gloss: 36.21, 	Dloss: 6.84
Episode 488	Total Average Score: -1207.57	Gloss: 36.29, 	Dloss: 6.84
Episode 489	Total Average Score: -1207.23	Gloss: 36.36, 	Dloss: 6.83
Episode 490	Total Average Score: -1204.05	Gloss: 36.43, 	Dloss: 6.84
Episode 491	Total Average Score: -1200.31	Gloss: 36.51, 	Dloss: 6.83
Episode 492	Total Average Score: -1196.32	Gloss: 36.58, 	Dloss: 6.82
Episode 493	Total Average Score: -1191.46	Gloss: 36.66, 	Dloss: 6.82
Episode 494	Total Average Score: -1198.45	Gloss: 36.73, 	Dloss: 6.81
Episode 495	Total Average Score: -1197.30	Gloss: 36.80, 	Dloss: 6.81
Episode 496	Total Average Score: -

### 5. Plot the result

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

plt.plot(np.arange(1, len(meanRs)+1), meanRs)
plt.ylabel('total_average_scores')
plt.xlabel('Episode #')
plt.show()

### 6. Watch the agent running with saved weights

In [14]:
# Load the saved weights into Pytorch model
agent.g.load_state_dict(torch.load('g-pendulum.pth', map_location='cpu'))
agent.d.load_state_dict(torch.load('d-pendulum.pth', map_location='cpu'))

# env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
# S = env_info.vector_observations                  # get the current states (S) (for each agent)
s = env.reset() # s: state # get the current state (s) (for an agent)

#Rs = np.zeros(num_agents)                          # initialize the total scores (Rs) (for each agent)
R = 0 # R:total reward # initialize the total score (R) (for an agent)

while True:
    #A = agent.act(S)                        # select actions (A) from loaded model agents
    a = agent.act(np.reshape(s, [1, -1])) # a: [-1, +1]    # select action (a) from loaded model agent
    
    # env_info = env.step(A)[brain_name]           # send all actions (A) to tne environment (env)
    # S2 = env_info.vector_observations         # get next states (S2) (for each agent)
    # rewards = env_info.rewards                         # get rewards (for each agent)
    # dones = env_info.local_done                        # see if the episode is done/finished (terminal)
    s2, r, done, _ = env.step(np.reshape(a, [-1]))
    
    # Rs += env_info.rewards                         # update the total scores (Rs) (for each agent)
    # S = S2                               # roll over current states (S) to next states (S2)
    R += r # update the total score (R) (for an agent)
    s = s2 # roll over current state (s) to next state (s2)
    
    #if np.any(dones):                                  # exit loop if episode is done/finished
    if done: # exit loop if episode is done/finished (terminal)
        break
        
print('Average of total scores: {}'.format(R))

FileNotFoundError: [Errno 2] No such file or directory: 'g-pendulum.pth'

When finished, you can close the environment.

In [6]:
env.close()